In [5]:
import os
import re
import logging
from parsing import CodeParser
from cache_utils import FunctionStateCache
from utils import load_cfile

logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("debug.log", 'w', encoding='utf-8'),
        logging.StreamHandler(),
    ]
)
logger = logging.getLogger(__name__)

pwd = os.getcwd()
fp = os.path.join(pwd, "data", "test1.c")
src_code = load_cfile(fp)

function_pattern = re.compile(r"\b(\w+)\s+(\w+)\s*\((.*?)\)\s*{")
variable_pattern = re.compile(r"\b(int|float|double|char)\s+(\w+)\s*=\s*([^;]+);")
argument_pattern = re.compile(r"\b(\w+)\s+(\w+)(?:\s*,\s*)?")
function_body_pattern = re.compile(r'\b\w+\s+\w+\s*\([^)]*\)\s*{([^{}]*(?:{[^{}]*}[^{}]*)*)}')

caches = FunctionStateCache()
parser = CodeParser(src_code, caches)
print()
caches._output_()

2024-11-11 23:38:13,682 - DEBUG - 
Function Name: functionA 
Return Type: void 
Arguments: {'x': 'int', 'y': 'int', 'z': 'int'} 
Content: 
 printf("In functionA with parameter x = %d\n", x); 

2024-11-11 23:38:13,683 - DEBUG - 
Function Name: main 
Return Type: int 
Arguments: {} 
Content: 
 printf("In main\n"); functionA(5, 6, 7); return 0; 

2024-11-11 23:38:13,684 - DEBUG - DEBUG MATCH: <re.Match object; span=(1, 8), match='printf('>
2024-11-11 23:38:13,684 - DEBUG - call func: printf(
2024-11-11 23:38:13,685 - DEBUG - call function: printf
2024-11-11 23:38:13,686 - WARNING - Failed to find function state `printf`.
2024-11-11 23:38:13,686 - DEBUG - DEBUG MATCH: <re.Match object; span=(9, 21), match='In main\\n");'>
2024-11-11 23:38:13,687 - DEBUG - statement: In main\n");
2024-11-11 23:38:13,687 - DEBUG - DEBUG MATCH: <re.Match object; span=(22, 32), match='functionA('>
2024-11-11 23:38:13,688 - DEBUG - call func: functionA(
2024-11-11 23:38:13,689 - DEBUG - call function: functionA


Function Name: functionA
Return Type: void
Arguments: {'x': 'int', 'y': 'int', 'z': 'int'}
Conditions: None
Content:  printf("In functionA with parameter x = %d\n", x); 

Function Name: main
Return Type: int
Arguments: {}
Conditions: None
Content:  printf("In main\n"); functionA(5, 6, 7); return 0; 



In [ ]:
import re

# Sample C code
c_code = """
printf("In main\\n"); functionA(5, 6, 7); return 0; 

int x = 5;
int y = 10;

if (x < y) {
    printf("x is less than y\\n");
} else {
    printf("x is not less than y\\n");
}

x = x + 1;
if (x == y) {
    printf("x is now equal to y\\n");
}
"""

# Define the patterns to capture if blocks, else blocks, and normal statements
patterns = {
    "if": r'\bif\s*\(.*?\)\s*{.*?}',
    "else": r'\belse\s*{.*?}',
    "statement": r'\b\w+\s+.*?;',
}

# Combine patterns to find all components in the order they appear
combined_pattern = re.compile(f"({patterns['if']})|({patterns['else']})|({patterns['statement']})", re.DOTALL)

# Parse the code by matching each component in sequence
def parse_code_in_order(c_code):
    for match in combined_pattern.finditer(c_code):
        if match.group(1):  # if block
            print("If block found:", match.group(1).strip())
        elif match.group(2):  # else block
            print("Else block found:", match.group(2).strip())
        elif match.group(3):  # normal statement
            print("Statement found:", match.group(3).strip())

# Parse the sample C code
parse_code_in_order(c_code)

In [ ]:
list(set(['8', '3', '6', '3']))

In [ ]:
condition_pattern = re.compile(r'if\s*\(([^)]*)\)')
code = """
void example() {
    if (x > 0 && y < 10) {
        doSomething();
    } else {
        doOther();
    }
}
"""

match = condition_pattern.search(code)
if match:
    condition = match.group(1)  # Gets "x > 0 && y < 10"
print(condition)

In [ ]:
import re

def extract_function_calls(c_statement):
    # Pattern matches function_name( with optional whitespace
    pattern = r'(\w+)\s*\('
    
    # Find all matches in the statement
    function_calls = re.findall(pattern, c_statement)
    
    return function_calls

# Example usage
statement = "result = calculate_sum(x, y);"
functions = extract_function_calls(statement)
print(f"Found function calls: {functions}")

In [ ]:
"1 -> 2".split(" -> ")